In [49]:
import pandas as pd

In [50]:
meteo = pd.read_csv("estaciones_control_datos_meteorologicos.csv", encoding="windows-1252", sep=";", skipfooter=10, engine="python", usecols=["CÓDIGO_CORTO", "DIRECCION"])
aq = pd.read_csv("informacion_estaciones_red_calidad_aire.csv", encoding="windows-1252", sep=";", usecols=["CODIGO_CORTO", "DIRECCION", "NOM_TIPO"])

In [51]:
meteo.rename({"CÓDIGO_CORTO": "CODIGO_CORTO"}, axis=1, inplace=True)

In [52]:
aq.head()

,CODIGO_CORTO,DIRECCION,NOM_TIPO
0,4,Plaza de España,Urbana tráfico
1,8,Entre C/ Alcalá y C/ O’ Donell,Urbana tráfico
2,11,Avda. Ramón y Cajal esq. C/ Príncipe de Vergara,Urbana tráfico
3,16,C/ Arturo Soria esq. C/ Vizconde de los Asilos,Urbana fondo
4,17,C/. Juan Peñalver,Urbana fondo


In [54]:
df = pd.merge(aq, meteo, "outer", on=["CODIGO_CORTO", "DIRECCION"]).fillna("Meteorologic")    

In [62]:
df.NOM_TIPO = df.NOM_TIPO.apply(lambda x: {"Urbana tráfico": "UrbanTraffic", 
                             "Suburbana": "Suburban", 
                             "Urbana fondo": "UrbanBackground", 
                             'Meteorologic': 'Meteorologic'}[x])

In [63]:
df

,CODIGO_CORTO,DIRECCION,NOM_TIPO
0,4,Plaza de España,UrbanTraffic
1,8,Entre C/ Alcalá y C/ O’ Donell,UrbanTraffic
2,11,Avda. Ramón y Cajal esq. C/ Príncipe de Vergara,UrbanTraffic
3,16,C/ Arturo Soria esq. C/ Vizconde de los Asilos,UrbanBackground
4,17,C/. Juan Peñalver,UrbanBackground
5,18,Calle Farolillo - C/Ervigio,UrbanBackground
6,24,Casa de Campo (Terminal del Teleférico),Suburban
7,27,"C/. Júpiter, 21 (Barajas)",UrbanBackground
8,35,Plaza del Carmen esq. Tres Cruces.,UrbanBackground
9,36,Avd. Moratalaz esq. Camino de los Vinateros,UrbanTraffic


In [72]:
df.to_csv("control_stations_info.csv", index=False)